In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from concurrent.futures import ThreadPoolExecutor
import itertools

In [2]:
url_list = [
    'https://www.mumsnet.com/talk/am_i_being_unreasonable/4676538-if-you-like-wordle-plusword-is-even-better-thread-4?page=',
    #'https://www.mumsnet.com/talk/_chat/4714295-plusword-new-thread-1?page=',
    #'https://www.mumsnet.com/talk/_chat/4765702-plusword-new-thread-2?page='
]

In [3]:
url_list

['https://www.mumsnet.com/talk/am_i_being_unreasonable/4676538-if-you-like-wordle-plusword-is-even-better-thread-4?page=']

In [4]:
url = url_list[0]

In [5]:
# maximum number of pages in thread
max_pages = 41

# html class of original post from the thread
original_post_class = 'p-4 pb-1 pt-2.5 lg:py-2.5 mt-2.5 lg:mt-1.5 border-t border-b sm:border sm:rounded ' \
                   'border-mumsnet-forest-border bg-mumsnet-forest dark:bg-mumsnet-forest-dark'

# html class of a normal post from the thread
normal_post_class = 'lg:py-2.5 pt-2.5 pb-1 p-4 border-t border-b sm:border sm:rounded mt-1.5 overflow-x-hidden ' \
                     'bg-white dark:bg-gray-800 border-gray-200'

# html class of a post from the thread creator
original_poster_reply_class = 'lg:py-2.5 pt-2.5 pb-1 p-4 border-t border-b sm:border sm:rounded mt-1.5 ' \
                              'overflow-x-hidden bg-mumsnet-forest dark:bg-mumsnet-forest-dark ' \
                              'border-mumsnet-forest-border'



In [6]:
all_posts = []

for page_number in range(1,39):

    # gets request via bs4

        r = requests.get(url + str(page_number))
        soup = BeautifulSoup(r.content, features="html5lib")

        post_list=[]
        posts_in_thread = []

        if page_number == 1 :

            original_post_list = []
            paragraph_list = []

            original_post = soup.find('div', class_=original_post_class)
            original_post = original_post.find_all('div', class_='')[2].find_all('p')

            for paragraph in original_post:
                post_text = paragraph.getText().split()
                post_text = ' '.join(post_text)
                paragraph_list.append(post_text)

            meta_data = paragraph_list[0].split()
            meta_data.pop(1)
            meta_data.append(paragraph_list[1])
            original_post = meta_data
            original_post_list.append(original_post)
            original_post_list = [val for sublist in original_post_list for val in sublist]

        posts = soup.find_all('div', class_=[normal_post_class, original_poster_reply_class])
        
        if posts is None:
            break
            
        del posts[0]

        for post in posts:
            post_text = post.getText().split()
            # separates out meta data and post body
            meta_data = post_text[:4]
            post_body = post_text[4:]

            # removes fullstop from meta data
            meta_data.pop(1)

            # converts whole of post body to one string
            post_body = ' '.join(post_body)
            meta_data.append(post_body)
            whole_post = meta_data
            post_list.append(whole_post)

        if  page_number== 1:
            post_list.insert(0, original_post_list)

        all_posts.append(post_list)
        
        continue
    
all_posts = [val for sublist in all_posts for val in sublist]

In [7]:
df = pd.DataFrame(all_posts, columns=['user', 'date', 'time', 'text'])
df

,user,date,time,text
0,ILoveAllRainbowsx,13/11/2022,14:18,Previous thread: www.mumsnet.com/talk/am_i_bei...
1,Drywhitefruitycidergin,13/11/2022,16:08,Thanks @ILoveAllRainbowsx - hopefully thread 4...
2,Readytostartagain,13/11/2022,16:25,Thanks for this @ILoveAllRainbowsx . Did CW in...
3,Hepzibar,13/11/2022,16:25,Thanks for the new thread and reminder @ILoveA...
4,Sunbird24,13/11/2022,16:29,@Readytostartagain sent a PM Add message Save ...
...,...,...,...,...
836,Readytostartagain,14/11/2022,08:07,1:42 today much easier than yesterday 😃 Add me...
837,UmbilicusProfundus,14/11/2022,08:09,⏱️ I just completed PlusWord in 01:16 slow sta...
838,MarmiteWine,14/11/2022,08:14,⏱️ I just completed PlusWord in 00:52 www.tele...
839,BrilliantGreenFlamingo,14/11/2022,11:57,6:19 today. Got a word wrong which threw me fo...


In [8]:
df.tail(50)

,user,date,time,text
791,Sunbird24,05/01/2023,00:18,⏱️ I just completed PlusWord in 00:38 No green...
792,JoyDivisionOvenGlovesx,05/01/2023,06:22,⏱️ I just completed PlusWord in 00:51 CW a fly...
793,DadDadDad,05/01/2023,07:16,1:25 for me - one of the easier ones. Add mess...
794,Stokey,05/01/2023,07:54,⏱️ I just completed PlusWord in 01:55 www.tele...
795,sanityisamyth,05/01/2023,08:09,1.09 for me. Yesterday's PW was a nightmare an...
796,BlandSoup,05/01/2023,16:14,The plusword I’ve got doesn’t make it work. I ...
797,BlandSoup,05/01/2023,16:17,Oops there was. Had the Italian composer spelt...
798,MarmiteWine,05/01/2023,19:05,⏱️ I just completed PlusWord in 01:09 www.tele...
799,Hepzibar,05/01/2023,19:27,⏱️ I just completed PlusWord in 01:47 www.tele...
800,bruffin,06/01/2023,00:06,⏱️ I just completed PlusWord in 01:59 www.tele...
